<a href="https://colab.research.google.com/github/brianhumphreys/Moire-Detector/blob/main/Copy_of_Playground_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Playground

In [34]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Fri Jul 16 02:55:01 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    33W / 250W |   8869MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [35]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('To enable a high-RAM runtime, select the Runtime > "Change runtime type"')
  print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
  print('re-execute this cell.')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


## Test 2D Wavelet Decomposition function
This function(fwdHaarDWT2D) computes the 2D Wavelet Transform in the image. All the input images are passed through a Haar Wavelet Decomposition module, to get the LL, LH, HL and HHH component of the image.

In [36]:
from google.colab import drive                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [37]:
# !ls /content/drive/MyDrive/Moire/train/negative/001/

In [38]:
# !ls /content/drive/MyDrive/Moire/trainDataNegative
# !ls /content/drive/MyDrive/Moire/train/negative/001
# !mv -v /content/drive/MyDrive/Moire/trainDataNegative/ '/content/drive/MyDrive/Moire/train/negative/001'
# !ls '/content/drive/MyDrive/Moire/train/negative/001/trainDataNegative'
# !mv -v '/content/drive/MyDrive/Moire/train/negative/001/trainDataNegative/*.tiff' '/content/drive/MyDrive/Moire/train/negative/001'
# import os

# fromDir = '/content/drive/MyDrive/Moire/notNormalizedPositiveImages/'
# toDir = "/content/drive/MyDrive/Moire/unnormalized/positive/002/"

# # !ls /content/drive/MyDrive/Moire/testDataNegative/

# for filename in os.listdir(fromDir):
#   os.rename(fromDir + filename, toDir + filename)

In [39]:
# import os
# os.chdir("/content/drive/")
# !ls

# os.chdir("../..")
# !ls


In [40]:
!ls /content/drive/MyDrive/Moire
!pip install pyheif whatimage
# !pip show wand

checkPoint  intermediate  preaugmented	test  train  unnormalized


In [41]:
# !ls /content/drive/MyDrive/Moire/notNormalizedPositiveImages/
# !ls /content/drive/MyDrive/Moire/notNormalizedNegativeImages/
# !uname -m

## Normalized Raw data to 1000x750 pixel images
Images are cropped to fit a 3:4 aspect ratio and then resized to match a 1000x750 size. Moves photos **unnormalized** -> **preaugmented**


In [42]:
# importing the module


import os
from os import listdir
from os.path import isfile, join
import numpy as np

import whatimage
import pyheif
from PIL import Image


def decodeImage(bytesIo):
    with open(bytesIo, 'rb') as f:
      data = f.read()
      fmt = whatimage.identify_image(data)
      if fmt in ['heic', 'avif']:
        i = pyheif.read_heif(data)
        pi = Image.frombytes(mode=i.mode, size=i.size, data=i.data)
        pi.save("heic.jpg", format="jpeg")

def read_heic(path: str):
    img = Wimage(path)
    img.format = 'jpg'
    img.save(filename="heic.jpg")
    img.close()

def openImage(fileName):
  decodeImage(fileName)
  return Image.open("heic.jpg")

def cropAndSave(image, fileName):
    width = image.size[0]
    height = image.size[1]

    aspect = width / float(height)

    if (height > width):
        image = image.rotate(90, Image.NEAREST, expand=1)
        width = image.size[0]
        height = image.size[1]

    ideal_width = 1000
    ideal_height = 750

    ideal_aspect = ideal_width / float(ideal_height)

    if aspect > ideal_aspect:
        # Then crop the left and right edges:
        new_width = int(ideal_aspect * height)
        offset = (width - new_width) / 2
        resize = (offset, 0, width - offset, height)
    else:
        # ... crop the top and bottom:
        new_height = int(width / ideal_aspect)
        offset = (height - new_height) / 2
        resize = (0, offset, width, height - offset)

    thumb = image.crop(resize).resize((ideal_width, ideal_height), Image.ANTIALIAS)
    thumb.save(fileName)



def normalizeRawImages(superpoch, dataSetNumber):

  print('##### NORMALIZING - superpoch: ' + superpoch + ' - dataset: ' + dataSetNumber)

  negativeFromDir = '/content/drive/MyDrive/Moire/unnormalized/negative/' + dataSetNumber + '/'
  positiveFromDir = '/content/drive/MyDrive/Moire/unnormalized/positive/' + dataSetNumber + '/'
  negativeToDir = '/content/drive/MyDrive/Moire/preaugmented/negative/' + dataSetNumber + '/'
  positiveToDir = '/content/drive/MyDrive/Moire/preaugmented/positive/' + dataSetNumber + '/'

  if not os.path.exists(positiveToDir):
      os.makedirs(positiveToDir)
  if not os.path.exists(negativeToDir):
      os.makedirs(negativeToDir)

  if len([name for name in os.listdir(positiveToDir)]) > 0:
    print('Directory ' + positiveToDir + ' already has normalized photos in it.  Skipping normalization for negative photos.')
  if len([name for name in os.listdir(negativeToDir)]) > 0:
    print('Directory ' + negativeToDir + ' already has normalized photos in it.  Skipping normalization for positive photos.')

  positiveImageFiles = [f for f in listdir(positiveFromDir) if (isfile(join(positiveFromDir, f)))]
  negativeImageFiles = [f for f in listdir(negativeFromDir) if (isfile(join(negativeFromDir, f)))]

  # LLList = [l for l in positiveImageFiles if 'LLL' in l]
  # print(LLList)

  if len([name for name in os.listdir(positiveToDir)]) <= 0:
    for f in positiveImageFiles:
        print(join(positiveFromDir, f))
        img = openImage(join(positiveFromDir, f))

        rgb_im = img.convert("RGB")
        components = f.split('.')
        newComponents = components[:len(components) - 1]
        newComponents.append('jpg')
        newFileName = '.'.join(newComponents)

        cropAndSave(rgb_im, join(positiveToDir, newFileName))

  if len([name for name in os.listdir(negativeToDir)]) <= 0:
    for f in negativeImageFiles:
        print(join(negativeFromDir, f))
        img = Image.open(join(negativeFromDir, f))

        rgb_im = img.convert("RGB")
        components = f.split('.')
        newComponents = components[:len(components) - 1]
        newComponents.append('jpg')
        newFileName = '.'.join(newComponents)

        cropAndSave(rgb_im, join(negativeToDir, newFileName))



In [43]:
#This function(fwdHaarDWT2D) computes the 2D Wavelet Transform in the image. All the input images are passed through a Haar Wavelet Decomposition module, to get the LL, LH, HL and HHH component of the image

import numpy as np
import pywt

def splitFreqBands(img, levRows, levCols):
    halfRow = int(levRows/2)
    halfCol = int(levCols/2)
    LL = img[0:halfRow, 0:halfCol]
    LH = img[0:halfRow, halfCol:levCols]
    HL = img[halfRow:levRows, 0:halfCol]
    HH = img[halfRow:levRows, halfCol:levCols]
    
    return LL, LH, HL, HH
    
def haarDWT1D(data, length):
    avg0 = 0.5;
    avg1 = 0.5;
    dif0 = 0.5;
    dif1 = -0.5;
    temp = np.empty_like(data)
    temp = temp.astype(float)
    h = int(length/2)
    for i in range(h):
        k = i*2
        temp[i] = data[k] * avg0 + data[k + 1] * avg1;
        temp[i + h] = data[k] * dif0 + data[k + 1] * dif1;
    
    data[:] = temp

# computes the homography coefficients for PIL.Image.transform using point correspondences
def fwdHaarDWT2D(img):
    img = np.array(img)
    levRows = img.shape[0];
    levCols = img.shape[1];
    img = img.astype(float)
    for i in range(levRows):
        row = img[i,:]
        haarDWT1D(row, levCols)
        img[i,:] = row
    for j in range(levCols):
        col = img[:,j]
        haarDWT1D(col, levRows)
        img[:,j] = col
        
    return splitFreqBands(img, levRows, levCols)

In [44]:
# from PIL import Image
# from matplotlib import pyplot as plt
# !ls /content/drive/MyDrive/Moire/
# img = Image.open('/content/drive/MyDrive/Moire/preaugmented/positive/001/IMG_2906.jpg').convert('L')
# img = img.transpose(Image.FLIP_LEFT_RIGHT)
# img.save('chl.jpg')
# LL, LH, HL, HH = fwdHaarDWT2D(img)
# fig, axes = plt.subplots(2, 2)
# fig.tight_layout()
# axes[0, 0].imshow(LL)
# axes[0, 1].imshow(LH)
# axes[1, 0].imshow(HL)
# axes[1, 1].imshow(HH)
# axes[0, 0].set_title("LL")
# axes[0, 1].set_title("LH")
# axes[1, 0].set_title("HL")
# axes[1, 1].set_title("HH")
# plt.show()

## Augment normalized data
The training images need to be put in two folders. positiveImages and negativeImages. positiveImages are the images which are captured from the display devices and has the presence of stron or weak Moiré patterms in it.
negativeImages are the ones without Moiré Patterns (i.e. the images which are not captured from the display devices).  Moves photos **preaugmented** -> **train/test** based on a split ratio.

In [45]:
import sys
import argparse
from PIL import Image
from PIL import ImageOps
import random
import sys
import os

from os import listdir
from os.path import isfile, join
from PIL import Image

#The training images need to be put in two folders. positiveImages and negativeImages. positiveImages are the images which are captured from the display devices and has the presence of stron or weak Moiré patterms in it. negativeImages are the ones without Moiré Patterns (i.e. the images which are not captured from the display devices)


def augmentNormalizedData(superpoch, dataSetNumber):

  print('##### AUGMENTING - superpoch: ' + superpoch + ' - dataset: ' + dataSetNumber)

  negativeFromDir = '/content/drive/MyDrive/Moire/preaugmented/negative/' + dataSetNumber + '/'
  positiveFromDir = '/content/drive/MyDrive/Moire/preaugmented/positive/' + dataSetNumber + '/'

  negativeToTrainDir = '/content/drive/MyDrive/Moire/train/negative/' + dataSetNumber + '/'
  positiveToTrainDir = '/content/drive/MyDrive/Moire/train/positive/' + dataSetNumber + '/'
  negativeToTestDir = '/content/drive/MyDrive/Moire/test/negative/' + dataSetNumber + '/'
  positiveToTestDir = '/content/drive/MyDrive/Moire/test/positive/' + dataSetNumber + '/'

  if not os.path.exists(negativeToTrainDir):
      os.makedirs(negativeToTrainDir)
  if not os.path.exists(positiveToTrainDir):
      os.makedirs(positiveToTrainDir)
  if not os.path.exists(negativeToTestDir):
      os.makedirs(negativeToTestDir)
  if not os.path.exists(positiveToTestDir):
      os.makedirs(positiveToTestDir)

  if len([name for name in os.listdir(negativeToTrainDir)]) > 0:
    print('Directory ' + negativeToTrainDir + ' already has normalized photos in it.  Skipping augmentation for negative photos.')
  if len([name for name in os.listdir(positiveToTrainDir)]) > 0:
    print('Directory ' + positiveToTrainDir + ' already has normalized photos in it.  Skipping augmentation for positive photos.')
  if len([name for name in os.listdir(negativeToTestDir)]) > 0:
    print('Directory ' + negativeToTestDir + ' already has normalized photos in it.  Skipping augmentation for negative photos.')
  if len([name for name in os.listdir(positiveToTestDir)]) > 0:
    print('Directory ' + positiveToTestDir + ' already has normalized photos in it.  Skipping augmentation for positive photos.')
        
  createTrainingData(positiveFromDir, negativeFromDir, positiveToTrainDir, negativeToTrainDir, positiveToTestDir, negativeToTestDir)

    
#The wavelet decomposed images are the transformed images representing the spatial and the frequency information of the image. These images are stored as 'tiff' in the disk, to preserve that information. Each image is transformed with 180 degrees rotation and as well flipped, as part of data augmentation.

def transformImageAndSave(image, f, customStr, path):
    cA, cH, cV, cD  = fwdHaarDWT2D(image);
    
    fileName = (os.path.splitext(f)[0])
    fLL = (f.replace(fileName, fileName+'_' + customStr + 'LL')).replace('.jpg','.tiff')
    fLH = (f.replace(fileName, fileName+'_' + customStr + 'LH')).replace('.jpg','.tiff')
    fHL = (f.replace(fileName, fileName+'_' + customStr + 'HL')).replace('.jpg','.tiff')
    fHH = (f.replace(fileName, fileName+'_' + customStr + 'HH')).replace('.jpg','.tiff')

    cA = Image.fromarray(cA)
    cH = Image.fromarray(cH)
    cV = Image.fromarray(cV)
    cD = Image.fromarray(cD)

    cA.save(join(path, fLL))
    cH.save(join(path, fLH))
    cV.save(join(path, fHL))
    cD.save(join(path, fHH))
    
    
def augmentAndTrasformImage(f, mainFolder, trainFolder):
    try:
        print(join(mainFolder, f))
        img = Image.open(join(mainFolder, f)) 
    except:
        print('Error: Couldnt read the file {}. Make sure only images are present in the folder'.format(f))
        return None

    imgGray = img.convert('L')
    wdChk, htChk = imgGray.size
    if htChk > wdChk:
        imgGray = imgGray.rotate(-90, expand=1)
        print('training image rotated')
    transformImageAndSave(imgGray, f, '', trainFolder)

    imgGray = imgGray.transpose(Image.ROTATE_180)
    transformImageAndSave(imgGray, f, '180_', trainFolder)

    imgGray = imgGray.transpose(Image.FLIP_LEFT_RIGHT)
    transformImageAndSave(imgGray, f, '180_FLIP_', trainFolder)
    
    return True
    
    
def createTrainingData(positiveFromDir, negativeFromDir, positiveToTrainDir, negativeToTrainDir, positiveToTestDir, negativeToTestDir):

    print('positive image path: ' + positiveFromDir)
    print('negative image path: ' + negativeFromDir)
    splitRatio = 0.8

    # get image files by classes
    positiveImageFiles = [f for f in listdir(positiveFromDir) if (isfile(join(positiveFromDir, f)))]
    negativeImageFiles = [f for f in listdir(negativeFromDir) if (isfile(join(negativeFromDir, f)))]

    positiveDataBorder = round(len(positiveImageFiles) * splitRatio)
    negativeDataBorder = round(len(negativeImageFiles) * splitRatio)

    positiveTrainFiles = positiveImageFiles[:positiveDataBorder]
    positiveTestFiles = positiveImageFiles[positiveDataBorder:]
    negativeTrainFiles = negativeImageFiles[:negativeDataBorder]
    negativeTestFiles = negativeImageFiles[negativeDataBorder:]

    print('positive train samples: ' + str(len(positiveTrainFiles)))
    print('negative train samples: ' + str(len(negativeTrainFiles)))
    print('positive test samples: ' + str(len(positiveTestFiles)))
    print('negative test samples: ' + str(len(negativeTestFiles)))

    Knegative = 0
    Kpositive = 0

    # create positive training images
    if len([name for name in os.listdir(positiveToTrainDir)]) <= 0:
      for f in positiveTrainFiles:
          ret = augmentAndTrasformImage(f, positiveFromDir, positiveToTrainDir)
          if ret == None:
              continue
          Kpositive += 3

    if len([name for name in os.listdir(negativeToTrainDir)]) <= 0:
      # create positive test images
      for f in negativeTrainFiles:
          ret = augmentAndTrasformImage(f, negativeFromDir, negativeToTrainDir)
          if ret == None:
              continue
          Kpositive += 3

    if len([name for name in os.listdir(positiveToTestDir)]) <= 0:
      # create negative training images
      for f in positiveTestFiles:
          ret = augmentAndTrasformImage(f, positiveFromDir, positiveToTestDir)
          if ret == None:
              continue
          Knegative += 3;

    if len([name for name in os.listdir(negativeToTestDir)]) <= 0:
      # create negative training images
      for f in negativeTestFiles:
          ret = augmentAndTrasformImage(f, negativeFromDir, negativeToTestDir)
          if ret == None:
              continue
          Knegative += 3;
    #
    # print('Total positive files after augmentation: ', Kpositive)
    # print('Total negative files after augmentation: ', Knegative)
    


# mainAugment('/content/drive/MyDrive/Moire/preaugmentedPositiveImages', '/content/drive/MyDrive/Moire/preaugmentedNegativeImages')



In [46]:
# from os import listdir
# from os.path import isfile, join
# from PIL import Image


# positiveImagePath = '/content/drive/MyDrive/Moire/preaugmentedPositiveImages'
# negativeImagePath = '/content/drive/MyDrive/Moire/preaugmentedNegativeImages'

# mainAugment(positiveImagePath, negativeImagePath, 0)
   

In [47]:
# !cd ../
# !ls

## Load Data into Memory
Get your big boy pants on because it's going to be a lot of data.  Increase runtime memory.  This section will load data from an **augmented** directory and loads it into a tensor for training or evaluation.  Recommended to make sure that there is no accelerator used so that it can be used when training.

In [48]:
from matplotlib import pyplot as plt
import numpy as np
import sys
import argparse
from os import listdir
from os.path import isfile, join
from PIL import Image
from sklearn import preprocessing
from sklearn.utils import shuffle
from skimage import io
from sklearn.model_selection import train_test_split
import os
from keras.utils import np_utils # utilities for one-hot encoding of ground truth values
from keras.callbacks import ModelCheckpoint

#constants
WIDTH = 500#384
HEIGHT = 375#512

#Here, we perform index based splitting and use those indices to split the our multi-input datasets. This is done because the CNN model is multi-input network
def splitTrainTestDataForBands(inputData, X_train_ind, X_test_ind):
    X_train = np.zeros((len(X_train_ind), WIDTH*HEIGHT))
    for i in range(len(X_train_ind)):
        X_train[i,:] = inputData[int(X_train_ind[i,0]),:]
        
    X_test = np.zeros((len(X_test_ind), WIDTH*HEIGHT))
    for i in range(len(X_test_ind)):
        X_test[i,:] = inputData[int(X_test_ind[i,0]),:]
        
    return X_train, X_test


def countPositiveSamplesAfterSplit(trainData):
    count = 0;
    for i in range(len(trainData)):
        if(trainData[i,0] == 0):
            count = count + 1
    return count

def scaleData(inp, minimum, maximum):
    minMaxScaler = preprocessing.MinMaxScaler(copy=True, feature_range=(minimum,maximum))
    inp = inp.reshape(-1, 1)
    inp = minMaxScaler.fit_transform(inp)
    
    return inp

def readAndScaleImage(f, customStr, trainImagePath, X_LL, X_LH, X_HL, X_HH, X_index, Y, sampleIndex, sampleVal):
    fileName = (os.path.splitext(f)[0])
    # print(fileName)
    # print(customStr)

    fLL = (f.replace(fileName, fileName + customStr)).replace('.jpg','.tiff')
    fLH = (f.replace(fileName, fileName + customStr)).replace('.jpg','.tiff')
    fHL = (f.replace(fileName, fileName + customStr)).replace('.jpg','.tiff')
    fHH = (f.replace(fileName, fileName + customStr)).replace('.jpg','.tiff')
    
    try:
        imgLL = Image.open(join(trainImagePath, fLL))
        imgLH = Image.open(join(trainImagePath, fLH))
        imgHL = Image.open(join(trainImagePath, fHL))
        imgHH = Image.open(join(trainImagePath, fHH))
    except Exception as e:
        print('Error: Couldnt read the file {}. Make sure only images are present in the folder'.format(fileName))
        print('Exception:', e)
        return None
        
    imgLL = np.array(imgLL)
    imgLH = np.array(imgLH)
    imgHL = np.array(imgHL)
    imgHH = np.array(imgHH)
    imgLL = scaleData(imgLL, 0, 1)
    imgLH = scaleData(imgLH, -1, 1)
    imgHL = scaleData(imgHL, -1, 1)
    imgHH = scaleData(imgHH, -1, 1)
    
    imgVector = imgLL.reshape(1, WIDTH*HEIGHT)
    X_LL[sampleIndex, :] = imgVector
    imgVector = imgLH.reshape(1, WIDTH*HEIGHT)
    X_LH[sampleIndex, :] = imgVector
    imgVector = imgHL.reshape(1, WIDTH*HEIGHT)
    X_HL[sampleIndex, :] = imgVector
    imgVector = imgHH.reshape(1, WIDTH*HEIGHT)
    X_HH[sampleIndex, :] = imgVector
    
    Y[sampleIndex, 0] = sampleVal;
    X_index[sampleIndex, 0] = sampleIndex;

    imgVector = None
    imgLL = None
    imgLH = None
    imgHL = None
    imgHH = None
    
    return True

def reshapeData(X_LL, X_LH, X_HL, X_HH, X_index, Y, imageCount):

    print('Dataset length: ' + str(len(X_LL)))
    
    print('num_train_samples', len(X_LL))
    X_LL = np.array(X_LL)
    X_LL = X_LL.reshape((len(X_LL), HEIGHT, WIDTH, 1))

    X_LH = np.array(X_LH)
    X_LH = X_LH.reshape((len(X_LH), HEIGHT, WIDTH, 1))

    X_HL = np.array(X_HL)
    X_HL = X_HL.reshape((len(X_HL), HEIGHT, WIDTH, 1))
    
    X_HH = np.array(X_HH)
    X_HH = X_HH.reshape((len(X_HH), HEIGHT, WIDTH, 1))

    Y = np.array(Y)
    
    return X_LL, X_LH, X_HL, X_HH, Y

def readImageSet(imageFiles, trainImagePath, X_LL, X_LH, X_HL, X_HH, X_index, Y, sampleIndex, bClass):

    for f in imageFiles:
        ret = readAndScaleImage(f, '', trainImagePath, X_LL, X_LH, X_HL, X_HH, X_index, Y, sampleIndex, bClass)
        if ret == True:
            sampleIndex = sampleIndex + 1

    return sampleIndex

def readWaveletData(positiveImagePath, negativeImagePath):
    
    # get augmented, balanced training data image files by class
    positiveImageFiles = [f for f in listdir(positiveImagePath) if (isfile(join(positiveImagePath, f)))]
    negativeImageFiles = [f for f in listdir(negativeImagePath) if (isfile(join(negativeImagePath, f)))]

    positiveCount = len(positiveImageFiles)
    negativeCount = len(negativeImageFiles)

    print('positive samples: ' + str(positiveCount))
    print('negative samples: ' + str(negativeCount))
    imageCount = positiveCount + negativeCount
    #intialization
    X_LL = np.zeros((positiveCount + negativeCount, WIDTH*HEIGHT))
    X_LH = np.zeros((positiveCount + negativeCount, WIDTH*HEIGHT))
    X_HL = np.zeros((positiveCount + negativeCount, WIDTH*HEIGHT))
    X_HH = np.zeros((positiveCount + negativeCount, WIDTH*HEIGHT))
    X_index = np.zeros((positiveCount + negativeCount, 1))
    Y = np.zeros((positiveCount + negativeCount, 1))
    
    sampleIndex = 0
    # read all images, convert to float, divide by 255 (leads to gray range 0..1), reshape into a row vector
    # write class 0 for positive and 1 for negative samples

    sampleIndex = readImageSet(positiveImageFiles, positiveImagePath, X_LL, X_LH, X_HL, X_HH, X_index, Y, sampleIndex, 0)
    print('positive data loaded.')
    
    sampleIndex += readImageSet(negativeImageFiles, negativeImagePath, X_LL, X_LH, X_HL, X_HH, X_index, Y, sampleIndex, 1)
    print('negative data loaded.')

    print('Total Samples Loaded: ', sampleIndex)
    
    X_LL, X_LH, X_HL, X_HH, Y = shuffle(X_LL, X_LH, X_HL, X_HH, Y, random_state=0)
    
    return X_LL, X_LH, X_HL, X_HH, X_index, Y, imageCount

def mainReadDatafromDrive(dataSetNumber, dataType):

    # dataType can be 'train' or 'test'

    positiveFromTrainDir = '/content/drive/MyDrive/Moire/' + dataType + '/positive/' + dataSetNumber + '/'
    negativeFromTrainDir = '/content/drive/MyDrive/Moire/' + dataType + '/negative/' + dataSetNumber + '/'

    if not os.path.exists(positiveFromTrainDir):
      print("ERROR: " + positiveFromTrainDir + ' does not exist.  Exiting.')
      raise ValueError('Directory does not exist')
    if not os.path.exists(negativeFromTrainDir):
      print("ERROR: " + negativeFromTrainDir + ' does not exist.  Exiting.')
      raise ValueError('Directory does not exist')
    
    X_LL, X_LH, X_HL, X_HH, X_index, Y, imageCount = readWaveletData(positiveFromTrainDir, negativeFromTrainDir)

    X_LL, X_LH, X_HL, X_HH, Y = reshapeData(X_LL, X_LH, X_HL, X_HH, X_index, Y, imageCount)

    return X_LL, X_LH, X_HL, X_HH, Y


## Train CNN Model

In [49]:
import os

from keras.models import Model # basic class for specifying and training a neural network
from keras.layers import Input, Convolution2D, MaxPooling2D, Dense, Dropout, Activation, Flatten, Add, Multiply, Maximum

def createModel(height, width, depth, num_classes):
#     num_epochs = 20 # 50 26 200 # we iterate 200 times over the entire training set
    kernel_size_1 = 7 # we will use 7x7 kernels 
    kernel_size_2 = 3 # we will use 3x3 kernels 
    pool_size = 2 # we will use 2x2 pooling throughout
    conv_depth_1 = 32 # we will initially have 32 kernels per conv. layer...
    conv_depth_2 = 16 # ...switching to 16 after the first pooling layer
    drop_prob_1 = 0.25 # dropout after pooling with probability 0.25
    drop_prob_2 = 0.5 # dropout in the FC layer with probability 0.5
    hidden_size = 32 # 128 512 the FC layer will have 512 neurons


    inpLL = Input(shape=(height, width, depth)) # depth goes last in TensorFlow back-end (first in Theano)
    inpLH = Input(shape=(height, width, depth)) # depth goes last in TensorFlow back-end (first in Theano)
    inpHL = Input(shape=(height, width, depth)) # depth goes last in TensorFlow back-end (first in Theano)
    inpHH = Input(shape=(height, width, depth)) # depth goes last in TensorFlow back-end (first in Theano)
    
    conv_1_LL = Convolution2D(conv_depth_1, (kernel_size_1, kernel_size_1), padding='same', activation='relu')(inpLL)
    conv_1_LH = Convolution2D(conv_depth_1, (kernel_size_1, kernel_size_1), padding='same', activation='relu')(inpLH)
    conv_1_HL = Convolution2D(conv_depth_1, (kernel_size_1, kernel_size_1), padding='same', activation='relu')(inpHL)
    conv_1_HH = Convolution2D(conv_depth_1, (kernel_size_1, kernel_size_1), padding='same', activation='relu')(inpHH)

    pool_1_LL = MaxPooling2D(pool_size=(pool_size, pool_size))(conv_1_LL)
    pool_1_LH = MaxPooling2D(pool_size=(pool_size, pool_size))(conv_1_LH)
    pool_1_HL = MaxPooling2D(pool_size=(pool_size, pool_size))(conv_1_HL)
    pool_1_HH = MaxPooling2D(pool_size=(pool_size, pool_size))(conv_1_HH)

    avg_LH_HL_HH = Maximum()([pool_1_LH, pool_1_HL, pool_1_HH])
    inp_merged = Multiply()([pool_1_LL, avg_LH_HL_HH])
    print(inp_merged)
    C4 = Convolution2D(conv_depth_2, (kernel_size_2, kernel_size_2), padding='same', activation='relu')(inp_merged)
    S2 = MaxPooling2D(pool_size=(4, 4))(C4)
    drop_1 = Dropout(drop_prob_1)(S2)
    C5 = Convolution2D(conv_depth_1, (kernel_size_2, kernel_size_2), padding='same', activation='relu')(drop_1)
    S3 = MaxPooling2D(pool_size=(pool_size, pool_size))(C5)
    C6 = Convolution2D(conv_depth_1, (kernel_size_2, kernel_size_2), padding='same', activation='relu')(S3)
    S4 = MaxPooling2D(pool_size=(pool_size, pool_size))(C6)
    drop_2 = Dropout(drop_prob_1)(S4)
    # Now flatten to 1D, apply FC -> ReLU (with dropout) -> softmax
    flat = Flatten()(drop_2)
    hidden = Dense(hidden_size, activation='relu')(flat)
    drop_3 = Dropout(drop_prob_2)(hidden)
    out = Dense(num_classes, activation='softmax')(drop_3)
    
    model = Model(inputs=[inpLL, inpLH, inpHL, inpHH], outputs=out) # To define a model, just specify its input and output layers
    intermediate_model = Model(inputs=[inpLL, inpLH, inpHL, inpHH], outputs=inp_merged)
    
    return model, intermediate_model

In [50]:
#To detect Moire ́ patternzs, images are first decomposed using Wavelet decomposition (refer to file '') and trained using multi-input Convolutional neural network. The strength of the proposed CNN model is, it uses the LL intensity image (from the Wavelet decomposition) as a weight parameter for the Moire ́ pattern, thereby approximating the spatial spread of the Moire ́ pattern in the image. Usage of CNN model performs better than frequency thresholding approach as the model is trained considering diverse scenarios and it is able to distinguish between the high frequency of background texture and the Moire ́ pattern.

from matplotlib import pyplot as plt
import numpy as np
import sys
import argparse
from os import listdir
from os.path import isfile, join
from PIL import Image
from sklearn import preprocessing
from sklearn.utils import shuffle
from skimage import io
from sklearn.model_selection import train_test_split
import os
from keras.utils import np_utils # utilities for one-hot encoding of ground truth values
from keras.callbacks import ModelCheckpoint
import time
from tensorflow import keras

# - read positive and negative training data
# - create X and Y from training data


def trainMoire(superpoch, dataSetNumber, numEpochs, weights_file = None):

    print('##### TRAINING - superpoch: ' + superpoch + ' - dataset: ' + dataSetNumber[0] + ', ', dataSetNumber[1])

    X_LL, X_LH, X_HL, X_HH, Y = mainReadDatafromDrive(dataSetNumber[0], 'train')
    X_LL_2, X_LH_2, X_HL_2, X_HH_2, Y_2 = mainReadDatafromDrive(dataSetNumber[1], 'train')


    print('Concatenating 2 datasets.')
    print(X_LL.shape)
    print(X_LL_2.shape)
    X_LL = np.concatenate((X_LL, X_LL_2), axis=0)
    X_LH = np.concatenate((X_LH, X_LH_2), axis=0)
    X_HL = np.concatenate((X_HL, X_HL_2), axis=0)
    X_HH = np.concatenate((X_HH, X_HH_2), axis=0)
    Y = np.concatenate((Y, Y_2), axis=0)
    X_LL_2 = None
    X_LH_2 = None
    X_HL_2 = None
    X_HH_2 = None
    Y_2 = None

    modelName = trainCNNModel(superpoch, dataSetNumber, X_LL, X_LH, X_HL, X_HH, Y, numEpochs, weights_file)
    
    X_LL = None
    X_LH = None
    X_HL = None 
    X_HH = None
    Y = None

    return modelName
    # evaluate(model, X_LL_test,X_LH_test,X_HL_test,X_HH_test,Y_test)
    

def trainCNNModel(superpoch, dataSetNumber, X_LL_train, X_LH_train, X_HL_train, X_HH_train, y_train, num_epochs, weights_file):

    batch_size = 32 # in each iteration, we consider 32 training examples at once
    print("SHAPE")
    print(X_LL_train.shape);
    num_train, height, width, depth = X_LL_train.shape
    num_classes = len(np.unique(y_train))
    Y_train = np_utils.to_categorical(y_train, num_classes) # One-hot encode the labels
    # Y_test = np_utils.to_categorical(y_test, num_classes) # One-hot encode the labels

    checkPointFolder = '/content/drive/MyDrive/Moire/checkPoint'
    checkpoint_name = checkPointFolder + '/mid-sp' + superpoch + '-ds' + dataSetNumber[0] + '_' + dataSetNumber[1] + '-ep{epoch:03d}-ls{val_loss:.5f}-ac{val_accuracy:.2f}-weights' 
    checkpoint = ModelCheckpoint(checkpoint_name, monitor='val_loss', verbose = 1, save_best_only = True, mode ='auto')
    callbacks_list = [checkpoint]
    
    if not os.path.exists(checkPointFolder):
        os.makedirs(checkPointFolder)
        
        
    model = None
    # if preloaded_model == None:
    #   print('A preloaded model was not provided.  Creating a new one')
    #   model, _ = createModel(height, width, depth, num_classes)

    if weights_file != None:
      print('--- Loading weights file: ' + weights_file)
      model = keras.models.load_model(weights_file)
    else:
      print('--- No weights file provided.  Compiling a new one.')
      
      model, _ = createModel(height, width, depth, num_classes)
      model.compile(loss='categorical_crossentropy', # using the cross-entropy loss function
                    optimizer='adam', # using the Adam optimiser
                    metrics=['accuracy']) # reporting the accuracy
      # if preloaded_model == None:
        
      # else:
      #   print('--- Model already loaded with weights.  Skipping compilation')

    model.fit([X_LL_train,X_LH_train,X_HL_train,X_HH_train], Y_train,                # Train the model using the training set...
              batch_size=batch_size, epochs=num_epochs,
              verbose=1, validation_split=0.1, callbacks=callbacks_list) # ...holding out 10% of the data for validation
    # score, acc = model.evaluate([X_LL_test,X_LH_test,X_HL_test,X_HH_test], Y_test, verbose=1)  # Evaluate the trained model on the test set!

    # print('------ weights ------')
    # for i in range(len(model.layers)):
    #   print(len(model.layers[i].get_weights()))

    modelName = '/content/drive/MyDrive/Moire/checkPoint/final-tm' + str(time.time()).split('.')[0] + '-sp' + superpoch + '-ds' + dataSetNumber[0] + '_' + dataSetNumber[1] + '-weights'
    model.save(modelName)
    
    return modelName


def evaluate(model, X_LL_test,X_LH_test,X_HL_test,X_HH_test,y_test):

    model_out = model.predict([X_LL_test,X_LH_test,X_HL_test,X_HH_test])
    passCnt = 0
    TP = 0
    TN = 0
    FP = 0
    FN = 0
    for i in range(len(y_test)):
        if np.argmax(model_out[i, :]) == y_test[i]:
            str_label='Pass'
            passCnt = passCnt + 1
        else:
            str_label='Fail'

        if y_test[i] ==0:
            if np.argmax(model_out[i, :]) == y_test[i]:
                TP = TP + 1;
            else:
                FN = FN + 1
        else:
            if np.argmax(model_out[i, :]) == y_test[i]:
                TN = TN + 1;
            else:
                FP = FP + 1

    start = "\033[1m"
    end = "\033[0;0m"
    print(start + 'confusion matrix (test / validation)' + end)
    print(start + 'true positive:  '+ end + str(TP))
    print(start + 'false positive: '+ end + str(FP))
    print(start + 'true negative:  '+ end + str(TN))
    print(start + 'false negative: '+ end + str(FN))
    print('\n')

    if TP+FP+FN+TN != 0:
      print(start + 'accuracy:  ' + end + "{:.4f} %".format(100*(TP+TN)/(TP+FP+FN+TN)))
    else:
      print(start + 'accuracy:  ' + end + "{:.4f} %".format(100))

    if TP + FP != 0:
      print(start + 'precision: ' + end + "{:.4f} %".format(100*TP/(TP + FP)))
    else:
      print(start + 'precision: ' + end + "{:.4f} %".format(100))

    if TP + FN != 0:
      print(start + 'recall:  ' + end + "{:.4f} %".format(100*TP/(TP + FN)))
    else:
      print(start + 'recall:  ' + end + "{:.4f} %".format(100))


In [51]:

# !ls /content/drive/MyDrive/Moire/checkPoint/
# from google.colab import files
# files.download('content/drive/MyDrive/Moire/checkPoint/Weights-028--0.06866.hdf5')

## Test CNN Evaluation

In [52]:
from matplotlib import pyplot as plt
import numpy as np
import sys
import argparse
from os import listdir
from os.path import isfile, join
from PIL import Image
from sklearn import preprocessing
from skimage import io
from sklearn.model_selection import train_test_split
import os

#constants
width = 500#384 #change dimensions according to the input image in the training
height = 375#512 #change dimensions according to the input image in the training
depth = 1
num_classes = 2

def testMoire(weightsFile, superpoch, dataSetNumber):

    print('##### TESTING - superpoch: ' + superpoch + ' - dataset: ' + dataSetNumber[0] + ', ' + dataSetNumber[1])
    X_LL, X_LH, X_HL, X_HH, Y = mainReadDatafromDrive(dataSetNumber[0], 'test')
    X_LL_2, X_LH_2, X_HL_2, X_HH_2, Y_2 = mainReadDatafromDrive(dataSetNumber[1], 'test')

    print('Concatenating 2 datasets.')
    X_LL = np.concatenate((X_LL, X_LL_2), axis=0)
    X_LH = np.concatenate((X_LH, X_LH_2), axis=0)
    X_HL = np.concatenate((X_HL, X_HL_2), axis=0)
    X_HH = np.concatenate((X_HH, X_HH_2), axis=0)
    Y = np.concatenate((Y, Y_2), axis=0)
    X_LL_2 = None
    X_LH_2 = None
    X_HL_2 = None
    X_HH_2 = None
    Y_2 = None
    
    CNN_model = keras.models.load_model(weightsFile)
    evaluate(CNN_model,X_LL,X_LH,X_HL,X_HH, Y)
    X_LL = None
    X_LH = None
    X_HL = None 
    X_HH = None
    Y = None

def run(model, X_LL_test,X_LH_test,X_HL_test,y_test):
    return


In [53]:
# weightsFile = "moirePattern3CNN_.h5"
    
    
# mainTest('content/drive/MyDrive/Moire/checkPoint/moirePattern3CNN_.h5', '/content/drive/MyDrive/Moire/testDataPositive', '/content/drive/MyDrive/Moire/testDataNegative')


## Extended Training
Take the trained model and continue training with additional data

In [54]:
# normalizeRawImages("001", "001")
# augmentNormalizedData("001", "001")
# normalizeRawImages("001", "002")
# augmentNormalizedData("001", "002")
# normalizeRawImages("001", "003")                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       001", "003")
# augmentNormalizedData("001", "003")
normalizeRawImages("001", "004")                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       001", "003")
# augmentNormalizedData("001", "004")

# weights_file = '/content/drive/MyDrive/Moire/checkPoint/mid-sp0010-ds003_001-ep005-ls0.00107-ac1.00-weights'
# weights_file = None

# superpoch_count = 10
# dataset_count = 4
# epochs = 5

# for sp in range(superpoch_count):
#   superpoch = '00' + str(sp + 1)
#   print('---------------------------------------------------------')
#   print('BEGINNING TRAINING OF SUPERPOCH: ' + superpoch)
#   print('---------------------------------------------------------')
#   for ds in range(dataset_count):
#     dsNumber = ds + 1
#     otherDsNumber = ((ds + 1) % dataset_count) + 1
#     datasets = ['00' + str(dsNumber), '00' + str(otherDsNumber)]

#     print(datasets)
#     # normalizeRawImages(superpoch, dataset)
#     # augmentNormalizedData(superpoch, dataset)

#     weights_file = trainMoire(superpoch, datasets, epochs, weights_file)
#     testMoire(weights_file, superpoch, datasets)
#     print('---------------------------------------------------------')
# /content/drive/MyDrive/Moire/preaugmented/positive/001/IMG_2866.HEIC
# /content/drive/MyDrive/Moire/preaugmented/positive/001/IMG_2875.HEIC
# /content/drive/MyDrive/Moire/preaugmented/positive/001/IMG_2893.HEIC
# /content/drive/MyDrive/Moire/preaugmented/positive/002/IMG_2900.HEIC
# /content/drive/MyDrive/Moire/preaugmented/positive/002/IMG_2904.HEIC
# /content/drive/MyDrive/Moire/preaugmented/positive/002/IMG_2915.HEIC
# /content/drive/MyDrive/Moire/preaugmented/positive/002/IMG_2921.HEIC
# /content/drive/MyDrive/Moire/preaugmented/positive/002/IMG_2929.HEIC



SyntaxError: ignored

In [ ]:
# X_LL_train, X_LH_train, X_HL_train, X_HH_train, Y_train = mainReadDatafromDrive('/content/drive/MyDrive/Moire/trainDataPositive', '/content/drive/MyDrive/Moire/trainDataNegative')
# !ls /content/drive/MyDrive/Moire/preaugmented/positive/003
print('negative unnormalized 001: ' + str(len([name for name in os.listdir('/content/drive/MyDrive/Moire/unnormalized/negative/001')])))
print('positive unnormalized 001: ' + str(len([name for name in os.listdir('/content/drive/MyDrive/Moire/unnormalized/positive/001')])))
print('negative unnormalized 002: ' + str(len([name for name in os.listdir('/content/drive/MyDrive/Moire/unnormalized/negative/002')])))
print('positive unnormalized 002: ' + str(len([name for name in os.listdir('/content/drive/MyDrive/Moire/unnormalized/positive/002')])))
print('negative unnormalized 003: ' + str(len([name for name in os.listdir('/content/drive/MyDrive/Moire/unnormalized/negative/003')])))
print('positive unnormalized 003: ' + str(len([name for name in os.listdir('/content/drive/MyDrive/Moire/unnormalized/positive/003')])))
# print('negative train 001: ' + str(len([name for name in os.listdir('/content/drive/MyDrive/Moire/train/negative/001')])))
# print('negative train 002: ' + str(len([name for name in os.listdir('/content/drive/MyDrive/Moire/train/negative/002')])))
# print('negative train 003: ' + str(len([name for name in os.listdir('/content/drive/MyDrive/Moire/train/negative/003')])))
# print('positive train 001: ' + str(len([name for name in os.listdir('/content/drive/MyDrive/Moire/train/positive/001')])))
# print('positive train 002: ' + str(len([name for name in os.listdir('/content/drive/MyDrive/Moire/train/positive/002')])))
# print('positive train 003: ' + str(len([name for name in os.listdir('/content/drive/MyDrive/Moire/train/positive/003')])))
# print('negative test 001: ' + str(len([name for name in os.listdir('/content/drive/MyDrive/Moire/test/negative/001')])))
# print('negative test 002: ' + str(len([name for name in os.listdir('/content/drive/MyDrive/Moire/test/negative/002')])))
# print('negative test 003: ' + str(len([name for name in os.listdir('/content/drive/MyDrive/Moire/test/negative/003')])))
# print('positive test 001: ' + str(len([name for name in os.listdir('/content/drive/MyDrive/Moire/test/positive/001')])))
# print('positive test 002: ' + str(len([name for name in os.listdir('/content/drive/MyDrive/Moire/test/positive/002')])))
# print('positive test 003: ' + str(len([name for name in os.listdir('/content/drive/MyDrive/Moire/test/positive/003')])))


In [ ]:
# testMoire('/content/drive/MyDrive/Moire/checkPoint/mid-sp005-ds002-ep016-ls0.00455-ac1.00-weights.h5', "001", "001")

In [ ]:
def intermediateMoire(dataSetNumber, weights_file = None):

    X_LL, X_LH, X_HL, X_HH, Y = mainReadDatafromDrive(dataSetNumber, 'test')
    return createIntermediateModel(dataSetNumber, X_LL, X_LH, X_HL, X_HH, Y, weights_file)
    

def createIntermediateModel(dataSetNumber, X_LL_train, X_LH_train, X_HL_train, X_HH_train, y_train, weights_file):

    batch_size = 32 # in each iteration, we consider 32 training examples at once
    print("SHAPE")
    print(X_LL_train.shape);
    num_train, height, width, depth = X_LL_train.shape
    num_classes = len(np.unique(y_train))
    Y_train = np_utils.to_categorical(y_train, num_classes) # One-hot encode the labels
    # Y_test = np_utils.to_categorical(y_test, num_classes) # One-hot encode the labels
    
    # if not os.path.exists(checkPointFolder):
    #     os.makedirs(checkPointFolder)
        
        
    model, intermediate_model = createModel(height, width, depth, num_classes)

    if weights_file != None:
      print('--- Loading weights file: ' + weights_file)
      model.load_weights(weights_file)
    else:
      print('--- No weights file provided.  Creating new model.')
    
    model.compile(loss='categorical_crossentropy', # using the cross-entropy loss function
                  optimizer='adam', # using the Adam optimiser
                  metrics=['accuracy']) # reporting the accuracy

    model.summary()

    # weights_list = model.get_weights()
    # for i in range(len(model.layers)):
    #   print(model.layers[i].get_weights())
      # print(model.layers[i].get_weights()[1])
    
    model_index = 0;
    print('----------------')
    for layer in intermediate_model.layers:
      for weights in layer.get_weights():
        weights = model.get_weights()[model_index]
        model_index = model_index + 1

    # print('----------------')

    # for layer in model.layers:
    #   for weights in layer.get_weights():
    #     print(len(weights))
    # print('----------------')
    
    # intermediate_model.layers[5].set_weights(weights_list[0])
    # print(len(model.layers))
    # intermediate_model.layers[i].set_weights(weights)

    # intermediate_model.summary()

    # model.fit([X_LL_train,X_LH_train,X_HL_train,X_HH_train], Y_train,                # Train the model using the training set...
    #           batch_size=batch_size, epochs=num_epochs,
    #           verbose=1, validation_split=0.1, callbacks=callbacks_list) # ...holding out 10% of the data for validation
    # score, acc = model.evaluate([X_LL_test,X_LH_test,X_HL_test,X_HH_test], Y_test, verbose=1)  # Evaluate the trained model on the test set!

    # modelName = '/content/drive/MyDrive/Moire/checkPoint/final-tm' + str(time.time()).split('.')[0] + '-sp' + superpoch + '-ds' + dataSetNumber + '-weights.h5'
    # model.save(modelName)
    
    return intermediate_model

In [ ]:
# from keras.models import Model

# model = intermediateMoire("001", '/content/drive/MyDrive/Moire/checkPoint/mid-sp005-ds002-ep016-ls0.00455-ac1.00-weights.h5')

# model.summary()
# X_LL, X_LH, X_HL, X_HH, Y = mainReadDatafromDrive("001", 'test')
# model_out = model.predict([X_LL, X_LH, X_HL, X_HH])


# print(model_out.shape)

In [ ]:
# from PIL import Image
# import numpy as np
# model_shape = model_out.shape

# reshaped_output = np.reshape(model_out, (model_shape[0], model_shape[3], model_shape[1], model_shape[2]))

# print(reshaped_output.shape)
# index = 0
# # for image_batch in reshaped_output:
# for image in reshaped_output[0]:
#   # image_batch = np.reshape(reshaped_output, (model_shape[0], model_shape[3], model_shape[1], model_shape[2]))
#   print(image.shape)

#   largest = 0
#   smallest = 100000

#   for row in image:
#     for i in row:
#       if i > largest:
#         largest = i
#       if i < smallest:
#         smallest = i

#   value_range = largest - smallest
#   scale_value = 255 / value_range

#   for i in range(len(image)):
#     for j in range(len(image[i])):
#       image[i][j] = scale_value * image[i][j]
#   print('largest: ' + str(largest))
#   print('smallest: ' + str(smallest))
#   im = Image.fromarray(image)
#   index = index + 1

#   if im.mode != 'RGB':
#     im = im.convert('RGB')

#   im.save('/content/drive/MyDrive/Moire/intermediate/test/negative/001/' + str(index) + ".jpg")

# print(index)
# print(np.reshape(model_out, (model_shape[0], model_shape[3], model_shape[1], model_shape[2])).shape)

In [ ]:
# #constants
# # width = 500#384 #change dimensions according to the input image in the training
# # height = 375#512 #change dimensions according to the input image in the training
# # depth = 1
# # num_classes = 2

# def continueTraining(weightsFile, X_LL_train, X_LH_train, X_HL_train, X_HH_train, Y_train, num_epochs):
#     weights_file = (weightsFile)

#     # X_LL, X_LH, X_HL, X_HH, Y = mainReadDatafromDrive(positiveImagePath, negativeImagePath)
    
    

#     batch_size = 32 # in each iteration, we consider 32 training examples at once
#     print("SHAPE")
#     print(X_LL_train.shape);
#     num_train, height, width, depth = X_LL_train.shape
#     num_classes = len(np.unique(Y_train))
#     Y_train = np_utils.to_categorical(Y_train, num_classes) # One-hot encode the labels
#     print(Y_train);
#     # Y_test = np_utils.to_categorical(y_test, num_classes) # One-hot encode the labels

#     CNN_model = createModel(height, width, depth, num_classes)
#     CNN_model.load_weights(weights_file)

#     checkPointFolder = 'content/drive/MyDrive/Moire/checkPoint'
#     checkpoint_name = checkPointFolder + '/Weights-retrained-002--{epoch:03d}--{val_loss:.5f}.hdf5' 
#     checkpoint = ModelCheckpoint(checkpoint_name, monitor='val_loss', verbose = 1, save_best_only = True, mode ='auto')
#     callbacks_list = [checkpoint]
    
#     if not os.path.exists(checkPointFolder):
#         os.makedirs(checkPointFolder)
        
        
#     # model = createModel(height, width, depth, num_classes)
    
#     CNN_model.compile(loss='categorical_crossentropy', # using the cross-entropy loss function
#                   optimizer='adam', # using the Adam optimiser
#                   metrics=['accuracy']) # reporting the accuracy

#     CNN_model.fit([X_LL_train,X_LH_train,X_HL_train,X_HH_train], Y_train,                # Train the model using the training set...
#               batch_size=batch_size, epochs=num_epochs,
#               verbose=1, validation_split=0.1, callbacks=callbacks_list) # ...holding out 10% of the data for validation
#     # score, acc = model.evaluate([X_LL_test,X_LH_test,X_HL_test,X_HH_test], Y_test, verbose=1)  # Evaluate the trained model on the test set!

#     CNN_model.save('content/drive/MyDrive/Moire/checkPoint/retrained-002-moirePattern3CNN_.h5')
    
#     return model
#     # evaluate(CNN_model,X_LL,X_LH,X_HL,X_HH, Y)
# import time
# str(time.time()).split('.')[0]

In [ ]:
# continueTraining('content/drive/MyDrive/Moire/checkPoint/Weights-028--0.06866.hdf5', X_LL_train, X_LH_train, X_HL_train, X_HH_train, Y_train, 1)

In [ ]:
# !ls content/drive/MyDrive/Moire/checkPoint

In [ ]:
# mainTest('content/drive/MyDrive/Moire/checkPoint/retrained-002-moirePattern3CNN_.h5', '/content/drive/MyDrive/Moire/testDataPositive', '/content/drive/MyDrive/Moire/testDataNegative')